In [1]:
# -*- coding: utf-8 -*-

"""
ensemble GA for binary class with n-Fold CV

"""
!pip install -q scikit-learn knnimpute scikit-survival
import os
import random
import statistics

import numpy as np
import scipy.stats as st
import time
import math
import sys

from copy import deepcopy
import argparse

import copy

from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from collections import Counter # for counting classes from labels list
from sklearn.metrics import fbeta_score, make_scorer # to make f2 for cross_validate
from scipy.spatial import distance

from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
from sklearn import svm
from joblib import Parallel, delayed
from joblib import dump, load

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

import itertools # for permutation of multi-labels
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler
from sksurv.metrics import concordance_index_censored

import joblib

        
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

## install from: https://pypi.org/project/knnimpute/
from knnimpute import (
    knn_impute_few_observed,
    knn_impute_with_argpartition,
    knn_impute_optimistic,
    knn_impute_reference,
)

"""
ensemble GA for binary class with n-Fold CV

"""
!pip install -q scikit-learn knnimpute scikit-survival
import os
import random
import statistics

import numpy as np
import scipy.stats as st
import time
import math
import sys

from copy import deepcopy
import argparse

import copy

from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from collections import Counter # for counting classes from labels list
from sklearn.metrics import fbeta_score, make_scorer # to make f2 for cross_validate
from scipy.spatial import distance

from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
from sklearn import svm
from joblib import Parallel, delayed
from joblib import dump, load

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

import itertools # for permutation of multi-labels
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler
from sksurv.metrics import concordance_index_censored

import joblib

        
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

## install from: https://pypi.org/project/knnimpute/
from knnimpute import (
    knn_impute_few_observed,
    knn_impute_with_argpartition,
    knn_impute_optimistic,
    knn_impute_reference,
)

In [2]:
import multiprocessing.managers
import copy
import concurrent.futures

In [3]:
import argparse

In [4]:
path = os.getcwd() #get cwd and append the file path to it  

# create output folder for pre-processing with timestamp to the path
tstamp = time.strftime('%Y_%m_%d')
output_folder='Output_PreProcess'+ os.sep+ str(tstamp) # get the path separator based on os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
pathProcess = os.path.join(path + os.sep, output_folder)

# create output folder for feature Selection with timestamp to the path
output_folder='Output_FeatSelect'+ os.sep+ str(tstamp) # get the path separator based on os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
pathFeature = os.path.join(path + os.sep, output_folder)

# create output folder for Model Saving with timestamp to the path
output_folder='Output_Model'+ os.sep+ str(tstamp) # get the path separator based on os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
pathModel = os.path.join(path + os.sep, output_folder)
'SEED CHANGE'
SEED = 1234 # seed for random state
'SEED CHANGE'
random.seed(1234) #Initialize random generator seed with the current local time
#NUM_CORES = cpu_count()
import psutil
NUM_CORES = psutil.cpu_count()

In [5]:
def load_model(file):
    remote_path = f"SADIQ/21 models/{file}"  
    !dx download "{remote_path}" --overwrite
    model = joblib.load(file)
    return model

In [6]:
cleaned_model_to_csvs = {
    "Sociodemographics": [
        "Sociodemographics"
    ],

    "Sociodemographics_oneDrop_qrisk": [
        "Sociodemographics",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history": [
        "Sociodemographics",
        "Health and medical history"
    ],

    "Sociodemographics_Health and medical history_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors"
    ],

    "Sociodemographics_Health and medical history_Sex-specific factors_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors"
    ],

    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history"
    ],

    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "Psychosocial factors"
    ],
    "Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "Psychosocial factors",
        "oneDrop"
    ],
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors_oneDrop":[
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "Psychosocial factors",
        "oneDrop"
    ],
    "Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "Psychosocial factors",
        "oneDrop",
        "qrisk"
    ],
    


}

In [7]:
def to_binary(x):                                 ### FIX – was missing
    """
    Normalise typical yes/no, true/false, 1/0 strings to 0 / 1.
    Leaves other values unchanged so pandas keeps dtypes.
    """
    if isinstance(x, str):
        s = x.strip().lower()
        if s in {"yes", "y", "true", "1"}:
            return 1
        if s in {"no", "n", "false", "0"}:
            return 0
    return x


In [8]:
def predictIndividual(model, columns_to_use, dfTestIndividual):
    # 1. Filter test DataFrame for required columns
    filtered_df = dfTestIndividual[
        [col for col in dfTestIndividual.columns if any(key in col for key in columns_to_use)]
        + ['clinicalrisk_Age.at.recruitment']
    ].copy()  

    # 2. Ensure correct feature order
    model_features = model.feature_names_in_

    for feature in model_features:
        if feature not in filtered_df.columns:
            filtered_df[feature] = 0  # Safe to modify now

    filtered_df = filtered_df[model_features]

    # 3. Return the risk score
    return model.predict(filtered_df)


In [9]:
def load_dataset(remote_path):
    import os
    import pandas as pd

    local_filename = os.path.basename(remote_path)  # extracts just 'dfTrain.csv'
    !dx download "{remote_path}" --overwrite
    dataset = pd.read_csv(local_filename)
    return dataset

dfTrain = load_dataset("SADIQ/30-04/dfTrain.csv")
dfTest = load_dataset("SADIQ/30-04/dfTest.csv")
dfTrain 

[===========================================================>] Completed 967,397,153 of 967,397,153 bytes (100%) /opt/notebooks/dfTrain.csvv
[===========================================================>] Completed 968,855,153 of 968,855,153 bytes (100%) /opt/notebooks/dfTest.csvv


,Unnamed: 0,eid,oneDrop_metabolomics_Apolipoprotein.B,oneDrop_metabolomics_Apolipoprotein.A1,oneDrop_metabolomics_Alanine,oneDrop_metabolomics_Glutamine,oneDrop_metabolomics_Glycine,oneDrop_metabolomics_Histidine,oneDrop_metabolomics_Total.Concentration.of.Branched.Chain.Amino.Acids..Leucine...Isoleucine...Valine.,oneDrop_metabolomics_Isoleucine,...,MEL_at_base,CRC_at_base,PC_at_base,BC_at_base,PD_at_base,OP_at_base,CAT_at_base,POAG_at_base,HT_at_base,testtrain
0,0,-1.000592,-0.313596,-1.083320,-0.317599,-0.774265,0.168655,-1.016457,-0.442465,0.200798,...,0,0,0,0,0,0,0,0,0,train
1,1,-1.000577,-0.260364,1.600931,-0.218882,0.122720,0.652646,0.621709,0.150410,-0.006246,...,0,0,0,0,0,0,0,0,0,train
2,5,-1.000543,0.724643,0.212878,-0.148123,0.167354,1.267813,-0.874707,0.025624,-0.043533,...,0,0,0,0,0,0,0,0,1,train
3,6,-1.000539,0.743573,-0.346315,0.702509,1.137018,-0.169060,0.863078,1.844256,2.094369,...,0,0,0,0,0,0,0,0,0,train
4,7,-1.000536,0.527013,0.528782,-0.346036,-0.342773,0.034109,-0.589745,-0.844637,-0.657025,...,0,0,0,0,0,0,0,0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120294,257225,1.000970,0.011964,0.025446,0.064152,0.242324,0.345130,-0.261630,0.698133,0.225687,...,0,0,0,0,0,0,0,0,0,train
120295,257227,1.000981,-0.547458,0.683941,-0.363845,-0.061498,-0.375194,1.805369,1.318727,1.213963,...,0,0,0,0,0,0,0,0,0,train
120296,257229,1.000988,0.501306,-0.301319,-0.632038,-0.200532,-0.735962,-0.212039,-0.325872,-0.267634,...,0,0,0,0,0,0,0,0,0,train
120297,257233,1.001018,0.438723,0.680217,-0.181540,-0.272936,-1.094466,1.705383,1.684035,1.512493,...,0,1,0,0,0,0,0,0,0,train


In [10]:
import pandas as pd

dfTrain['followup_bin'] = pd.qcut(dfTrain['CVD_followup'], q=10, duplicates='drop')
'SEED CHANGE'

sampled_df = dfTrain.groupby('followup_bin', group_keys=False).apply(
    lambda x: x.sample(frac=5000/len(dfTrain), random_state=1234)
)

sampled_df = sampled_df.drop(columns='followup_bin')

In [11]:
print("Original dfTrain:")
print(dfTrain['CVD_followup'].describe())

print("\nSampled df:")
print(sampled_df['CVD_followup'].describe())

Original dfTrain:
count    120299.000000
mean         12.828773
std           2.811520
min           0.002738
25%          12.668036
50%          13.596167
75%          14.362765
max          15.865845
Name: CVD_followup, dtype: float64

Sampled df:
count    4999.000000
mean       12.836918
std         2.783891
min         0.019165
25%        12.665298
50%        13.596167
75%        14.364134
max        15.542779
Name: CVD_followup, dtype: float64


In [12]:
len(sampled_df.columns)

1625

In [13]:
len(dfTrain.columns)

1626

In [14]:
dfTrain = sampled_df

In [15]:
len(dfTrain.columns)

1625

In [16]:
len(dfTrain)

4999

In [17]:
print(len(dfTest))

# Drop rows where CVD_followup is between 10 and 12 AND CVD_status is True
dfTest = dfTest.drop(
    dfTest[
        (dfTest['CVD_followup'] >= 10) &
        (dfTest['CVD_followup'] <= 12) &
        (dfTest['CVD_status'] == True)
    ].index
)

print(len(dfTest))

print((dfTest['CVD_status'] == True).sum())


120495
118376
10706


In [19]:
models = [
    "MRMR_COX_Sociodemographics.pkl",
    "MRMR_COX_Sociodemographics_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_oneDrop_qrisk.pkl",
    
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors.pkl",
    "MRMR_COX_Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop.pkl",
    "MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk.pkl"
    
    
]

In [20]:
modelsPickleList = []
for i in range(0,len(models)):
    modelsPickleList.append(load_model(models[i]))
    
print(len(modelsPickleList))
    

[===========================================================>] Completed 140,774 of 140,774 bytes (100%) /opt/notebooks/MRMR_COX_Sociodemographics.pkll
[===========================================================>] Completed 151,715 of 151,715 bytes (100%) /opt/notebooks/MRMR_COX_Sociodemographics_oneDrop.pkll
[===========================================================>] Completed 153,413 of 153,413 bytes (100%) /opt/notebooks/MRMR_COX_FULL_MODEL_+QRISKSociodemographics_oneDrop_qrisk.pkll
[===========================================================>] Completed 155,785 of 155,785 bytes (100%) /opt/notebooks/MRMR_COX_Sociodemographics_Health_and_medical_history.pkll
[===========================================================>] Completed 166,726 of 166,726 bytes (100%) /opt/notebooks/MRMR_COX_Sociodemographics_Health_and_medical_history_oneDrop.pkll
[===========================================================>] Completed 168,424 of 168,424 bytes (100%) /opt/notebooks/MRMR_COX_FULL_MODEL

In [21]:
def evaluateBranch(cleanModel, cleanModelColumns,row, bloodModel, bloodModelColumns,ind1,ind2):
    predictedOutput = predictIndividual(cleanModel, cleanModelColumns, row)[0]
    if predictedOutput > ind2:
        'high risk '
        predictedOutput = predictIndividual(bloodModel, bloodModelColumns, row)[0]
        return predictedOutput
    elif predictedOutput < ind1:
        'low risk'
        return predictedOutput
    else:
        return 'not right'
    

In [22]:
print("Original dfTrain:")
print(dfTrain['CVD_followup'].describe())

print("\nSampled df:")
print(sampled_df['CVD_followup'].describe())

Original dfTrain:
count    4999.000000
mean       12.836918
std         2.783891
min         0.019165
25%        12.665298
50%        13.596167
75%        14.364134
max        15.542779
Name: CVD_followup, dtype: float64

Sampled df:
count    4999.000000
mean       12.836918
std         2.783891
min         0.019165
25%        12.665298
50%        13.596167
75%        14.364134
max        15.542779
Name: CVD_followup, dtype: float64


In [23]:
dfTestFeaturesCols = dfTest.columns

In [24]:

######### feature selection class ###############################################
##################################################################################
class DiagnosticOptimization:

    # Constructor, called at the start of this class
    #def __init__(self,dataset, proteins,min_values, max_values,class_Labels, evaluate_values,categories,
    #             population, generations, folds):
    def __init__(self,dataset, proteins,class_Labels,testset_dataset, testset_class_Labels, evaluate_values,population, generations):
        # variables for these parameters
        self.dataset = dataset
        self.testset_dataset = testset_dataset
        self.proteins = proteins
        self.class_Labels = class_Labels # labels or classes as list of list from pre process step
        self.testset_class_Labels = testset_class_Labels # labels or classes as list of list from pre process step
        self.powerSetClass = [] # initialise powerset class labels
        self.evaluate_values = evaluate_values # count of parameters to be optimised
        
        self.min_values = [-2.5,5,-2.5,5,-2.5,5,-2.5,5,-2.5,5,-2.5,5,-2.5,5,-2.5,0] # min value for parameters to be optimised
        self.max_values = [1,9,1,9,1,9,1,9,1,9,1,9,1,9,1,5] # max value for parameters to be optimised
                
        self.population = int(population) # population size in GA
        self.generations = int(generations) # no of generations in GA
        self.twoCross_prob = 0.45 # 2-point cross over probability
        self.arithCross_prob = 0.45 # arithmetic cross over probability
        self.mutation_prob = [0.001,0.2] # mutation probabilityy, min = 0.001 and max = 0.2
        self.gaussMut_varPro = [0.1,0.5] # gaussian variance proportion, min=0.1 and max = 0.5
        self.avgSimThr = 0.90 # average similariy between the members of the population and its best member
                
        
        
        
        
        self.output_message = '' # output message from feature selection
        self.goal_header = "Questions_proportion,Single_blood_proportion,Risk_scores_proportion,True_positive_rate,False_negative_rate,AUC,Cindex,Weighted_Sum"
        # goal significance for goals position 0 thru 8; includes distance. function getParetoFront() ParetoIndex to be aligned with weightage in goal_significance here
        self.goal_significance = [1.0, 1.0, 1.0,1.0,1.0,1.0, 1.0] # weightage for metrics 
        self.convergeThreshold = 0.001 # convergence threshold for GA
        
   
        # variables for goal values (post pareto front goal tuning) 
        self.max_eval_per_generation=[0]*self.generations # best values per goal per generation
        self.average_eval_per_generation=[0]*self.generations # weighhted avg value per generation across all goals
        self.sum_ranked_eval_per_generation=[0]*self.generations # Sum of weighted average goal per generation
        self.average_ranked_eval_per_generation=[0]*self.generations # Avg of weighted average goal per generation
        self.max_sol_per_generation=[0]*self.generations # best sol per generation
        self.Pareto_Wheel_Output = '' # output message from Pareto front and roulette wheel
    
    # Accessor functions for parameters
    def setPopulation(self, value):
        self.population = value

    def getPopulation(self):
        return self.population

    def setGenerations(self, value):
        self.generations = value

    def getGenerations(self):
        return self.generations



    #This function initializes a population of solutions (float vectors) for optimisation parameters
    # This function is called by biomarker_feature_selection()
    def initialize(self):
        population = self.getPopulation() #population:(integer) it is the number of individual solutions which evolve in parallel
        min_threshold = 0.3
        min_values=list()
        max_values= list() 
        
        for i in range(15):
            if i%2==0:
                #stop criterion low risk #SADIQ to confirm this is in months or in years (how it appears) In principle we want to have a stopping criterion if the model predict very high score (very close event)
                min_values.extend([-2.5])
                max_values.extend([1])
                #max_values.extend([16])
            else:
                #stop criterion high risk #SADIQ to confirm this is in months or in years (how it appears) In principle we want to have a stopping criterion if the model predict very low score (very distant event)
                min_values.extend([5])
                max_values.extend([9])
        min_values.extend([0])
        max_values.extend([5])
        
        # initialise population count (size) as list of list with each individual =  parameters + feature length 
        individuals=[[0 for x in range(len(min_values))] for x in range(population)] 
        
        # generate random values for each individual in the population
        for i in range(len(individuals)): # iterate on population size i.e. count of individuals
            for j in range(0, len(min_values)): # iterate on size of each individual
                individuals[i][j] = random.uniform(min_values[j],max_values[j]) # assign values between min and max at random
        
        self.output_message += "Population initialised for %s"%len(individuals) + " individuals \n"
                        
        return individuals
    
    #This function initializes a population of solutions (float vectors) with front1 solutions of previous run saved as FinalSolutionsAll.csv
    # this step needs to be manually activated and is not automated to pick previous run final solution into next run
    # This function is called by biomarker_feature_selection()
    
    # function to split data by class; used by wilconxon ranksum for a specific feature index
    def splitData_perFeat_byClass(self, feature_index,lblClassCount,lbl):
        
        listClassData = [] # initialise list of list to hold data by class for a given feature index
        classIndex = 0
        # split the dataset into class labels for each feature
        for k in lblClassCount.keys():
            listClassData.append([])
            for j in range(len(self.dataset[0])): # segregate observations per protein i.e. feature
                if int(self.class_Labels[lbl][j])==int(k):
                    listClassData[classIndex].append(float(self.dataset[feature_index][j]))
            classIndex = classIndex +1 # increment the counter for class
        
        return listClassData # returns list of list    
    # function to get individual class and ovreall ROC for multiclass problem
    def roc_multiclass(self,X_train,X_test,y_train, y_test,classifier_type,clf):
        
        # get Int labels as self.dictCountLabels has floats; 
        #gamma_val,C_val,TreeCount
        # Dictionary of labels and associated count of samples
        dictIntLabels = {int(k):v for k,v in self.dictCountLabels.items()}
        
        # extract keys from dictionary and hold as list of classes
        class_label=[]
        class_support = []
        for key,values in dictIntLabels.items():
            class_label.append(key)
            class_support.append(values) # support for each class
                
        # Binarize the output
        ynew_test = label_binarize(y_test, classes=sorted(class_label))
        ynew_train = label_binarize(y_train, classes=sorted(class_label))
        n_classes = ynew_train.shape[1]
        
        clf.fit(X_train, ynew_train) # fit on train data
        
        y_score = clf.decision_function(X_test) # for SVM; for RF instead used predict_proba(); can be automated with if condition on clf type
        
        # Compute ROC AUC for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(ynew_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        
        # Calculate simple average i.e. micro roc
        i=0 # Counter to fetech suport as class labels not necessarilt 0,1,2
        roc_average = []
        for key, values in roc_auc.items():
            roc_average.append(values)
            i=i+1

        return (sum(roc_average)/i),roc_average # return average and roc for each class as list


    '''# function to calculate remaining goal values (as per the fitness function) for each of the individual solutions in a given population
    # goal_values inialised to zero and calculated in this function
    # This function is called by evaluate_indivdiuals() for each individual solution. final_selected is features post intersection with GA.
    # classifier_type - is the value of the 1st position of the individual solution that decides the classifier type'''
    def train_Classifier(self, dataTran,final_selected,classifier_type, C_val, Gamma_val,Classchain,TreeCount,ftwo_scorer,finalFlag,multilabel):
        inputs = [] # initialise X of the model
        Colindices = [] # feature indices selected
        outputs = np.ravel(np.array(deepcopy(self.class_Labels)).T) # assign to a variable so that synthetic data labels can be apepnded
        # Create classification model dataset aligned with features selected
        # X i.e. input to be of the form shape [n_samples, n_features]
        # dataTRan is np.ndarray with shape as required by X
        for feature in range(len(final_selected)):
            if final_selected[feature]==1: # append only those feature that are 1 i.e. selected
                Colindices.append(feature) # get the indices of features selected
        
        self.output_message += "Feature indices in final selection : %s"%Colindices + "\n"
        # append all rows i.e. observations for the feature selected
        inputs = dataTran[:,Colindices]  # type = numpy array
        
        #######################################################################
        ###################### Choose the classifier ##########################
        print ("training classifiers")
        
        SV_Tree = 0.0001 # assign insignificantly low value so that goal value is not zero
        Dist_Metric = 0.0001
        if classifier_type <1.0: #linear svm
            clf = OneVsRestClassifier(svm.SVC(C=C_val, kernel='linear', random_state=SEED))
        elif classifier_type >=1.0 and classifier_type <2.0: #RBF svm
            #for multi-class use OneVsRestClassifier as SVC does ovo and use descion function 
            #https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html #sklearn.svm.SVC
            # if class_weight="balanced" is used then use oversampling; 
            #default gamma is being used; to optimize via GA use Gamma_val
            clf = OneVsRestClassifier(svm.SVC(C=C_val, gamma= 'scale', kernel='rbf',random_state=SEED)) #, random_state=SEED
            #for binary SVC can be used
            #clf = svm.SVC(C=C_val, gamma= 'scale', kernel='rbf', random_state=SEED)
            
        else: # random forest, n_jobs=-1 for parallelsim; 
            #although RF inherently multi-class, OneVsRestClassifier may b better for convex loss
            # additional parameter required for tree depth or use variable Gamma_val if unused for SVM with defualt usage 
            clf = OneVsRestClassifier(RandomForestClassifier(n_estimators = int(TreeCount), oob_score=True, random_state=SEED,n_jobs=-1))
            SV_Tree = float((self.max_trees - int(TreeCount))/self.max_trees) # don't round so that insignificantly low values are kept
            
        #### flag to check if this is for final model saving or evaluation of solutions during GA
        if finalFlag == True: # if final model being trained then no CV required
            inputs, outputs = self.sampling_function(outputs,inputs,multilabel) # perform oversampling of imbalanced class
            
            clf.fit(inputs,outputs) # train on all data
            print ("final model fitting done")
            return clf, inputs, outputs
        
        #######################################################################################
        ###################### Run CV and get the performance scores ##########################
        skf = StratifiedKFold(n_splits=self.getFolds()) #  stratified to preserve the percentage of samples for each class.
        sv_list=[]
        dist_list = []
        
        # initailise performance metrics to compute agreegate at fold level
        accuracy_fold = [] # this measures accuracy as all labels predicted correct 
        
        y_true = [] # to append the held-out cases for final ROC computation
        y_pred_Class1prob = [] # to append the held-out cases for final ROC computation
        y_pred_Class=[]
        npv_split=[]
        specificity_fold = []
        precision_fold = []
        recall_fold = []
        roc_fold = []
        tn_all=0
        fp_all=0
        fn_all=0
        tp_all=0
        
        cv_scores = [] # initialise list of performance scores averaged over k-folds
        
        # CV Loop coded only for binary classification problem
        for train_index, test_index in skf.split(inputs, outputs): # takes 20% as test data
            X_train, X_test = inputs[train_index,:], inputs[test_index,:] # inputs is ndarray so use np.ndarray slicing
            y_train, y_test = np.array(outputs)[train_index], np.array(outputs)[test_index]
            
            X_train, y_train = self.sampling_function(y_train,X_train,multilabel) # MTD oversampling of all classes
            
            clf.fit(X_train, y_train) # fit on train data
            
            # 'OneVsRestClassifier' object has no attribute 'n_support_'
            if classifier_type <2.0: sv_list.append(0.0001)
            
            y_pred = clf.predict(X_test)
            if classifier_type <2.0: # svm
                y_score = clf.decision_function(X_test)
                y_pred_Class1prob.append(clf.decision_function(X_test)) # get deciison function
            else: # RF
                y_score = clf.predict_proba(X_test) # for RF use probaility;
                y_pred_Class1prob.append(y_score[:,1])
                
            y_true.append(y_test)
            y_pred_Class.append(y_pred)
            
            accuracy_fold.append(accuracy_score(y_test, y_pred)) # to get distance or accuracy based on hamming loss
            specificity_fold.append(recall_score(y_test, y_pred, pos_label=0, average='binary')) # true negative rate or specificity
            precision_fold.append(precision_score(y_test, y_pred, average='binary'))
            recall_fold.append(recall_score(y_test, y_pred, average='binary'))
            if classifier_type <2.0: # svm
                roc_fold.append(roc_auc_score(y_test,y_score)) # y_score[:,1]
            else: # RF
                roc_fold.append(roc_auc_score(y_test,y_score[:,1])) # y_score[:,1]
            
            #npv calculate
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() 
            tn_all = tn_all+tn
            fp_all = fp_all+fp
            fn_all = fn_all+fn
            tp_all = tp_all+tp
            
            if (tn>=0 and fn>0) or (tn>0 and fn>=0):
                npv= tn/(tn+fn)
            else: npv = 0.0
            npv_split.append(npv)
            
            err = 0.0
            for err_index in range(len(X_test)):
                if y_pred[err_index] != y_test[err_index]: 
                    err = err + 1.0
            # store the k-fold error and accuracy of the ensemble model
            dist_list.append(1.0 - float(err/len(X_test)))
        
        if classifier_type <2.0: SV_Tree = np.mean(np.array(sv_list))
        Dist_Metric = np.mean(np.array(dist_list))
        
        # take average of metrics at fold level and append in sequence 
        # [Accuracy,NPV, Specificity, PPV,Recall,Roc_auc]
        cv_scores.append(np.mean(np.array(accuracy_fold)))
        cv_scores.append(np.mean(np.array(npv_split)))
        cv_scores.append(np.mean(np.array(specificity_fold)))
        cv_scores.append(np.mean(np.array(precision_fold)))
        cv_scores.append(np.mean(np.array(recall_fold)))
        cv_scores.append(np.mean(np.array(roc_fold)))
        
        return cv_scores, SV_Tree,Dist_Metric
    
    '''#This function is used to calculate the optimization goals and is called by optimization()
    # Inputs are single individual solution, pVal for features calculated frm wilcoxon & transpose data for Kbest & MI feature selection
    # goal_values inialised to zero for single sol and calculated in this function'''
    
    def evaluate_individuals(self,ind, dataset): 

        Questions_proportion = 0
        Single_blood_proportion = 0 
        Risk_scores_proportion = 0 
        predicted_output = [0 for _ in range(len(dataset))]

        actualValuesStatus = []
        actualValuesFollowup = []
        
        for i in range(len(dataset)):
            patient = dataset.iloc[[i]]
            actualValuesStatus.append(dataset['CVD_status'].iloc[i])
            actualValuesFollowup.append(dataset['CVD_followup'].iloc[i])

            predicted = evaluateBranch(
                modelsPickleList[0],
                cleaned_model_to_csvs["Sociodemographics"],
                patient,
                modelsPickleList[2],
                cleaned_model_to_csvs["Sociodemographics_oneDrop_qrisk"],
                ind[0],
                ind[1]
            )
            
            if isinstance(predicted, float) and predicted < ind[0]:
                predicted_output[i] = predicted
                Questions_proportion += 97


            elif isinstance(predicted, float) and predicted > ind[1]:
                predicted_output[i] = predicted
                Questions_proportion += 97
                Single_blood_proportion += 185
                Risk_scores_proportion += 44

            else:
                predicted = evaluateBranch(
                    modelsPickleList[3],
                    cleaned_model_to_csvs["Sociodemographics_Health and medical history"],
                    patient,
                    modelsPickleList[5],
                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_oneDrop_qrisk"],
                    ind[2],
                    ind[3]
                )

                if isinstance(predicted, float) and predicted < ind[2]:
                    predicted_output[i] = predicted
                    Questions_proportion += 242

                elif isinstance(predicted, float) and predicted > ind[3]:
                    predicted_output[i] = predicted
                    Questions_proportion += 242
                    Single_blood_proportion += 185
                    Risk_scores_proportion += 44

                else:
                    predicted = evaluateBranch(
                        modelsPickleList[6],
                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors"],
                        patient,
                        modelsPickleList[8],
                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_oneDrop_qrisk"],
                        ind[4],
                        ind[5]
                    )

                    if isinstance(predicted, float) and predicted < ind[4]:
                        predicted_output[i] = predicted
                        Questions_proportion += 312


                    elif isinstance(predicted, float) and predicted > ind[5]:
                        predicted_output[i] = predicted
                        Questions_proportion += 312
                        Single_blood_proportion += 185
                        Risk_scores_proportion += 44

                    else:
                        predicted = evaluateBranch(
                            modelsPickleList[9],
                            cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors"],
                            patient,
                            modelsPickleList[11],
                            cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_oneDrop_qrisk"],
                            ind[6],
                            ind[7]
                        )

                        if isinstance(predicted, float) and predicted < ind[6]:
                            predicted_output[i] = predicted
                            Questions_proportion += 319


                        elif isinstance(predicted, float) and predicted > ind[7]:
                            predicted_output[i] = predicted
                            Questions_proportion += 319
                            Single_blood_proportion += 185
                            Risk_scores_proportion += 44

                        else:
                            predicted = evaluateBranch(
                                modelsPickleList[12],
                                cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history"],
                                patient,
                                modelsPickleList[14],
                                cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_oneDrop_qrisk"],
                                ind[8],
                                ind[9]
                            )

                            if isinstance(predicted, float) and predicted < ind[8]:
                                predicted_output[i] = predicted
                                Questions_proportion += 376


                            elif isinstance(predicted, float) and predicted > ind[9]:
                                predicted_output[i] = predicted
                                Questions_proportion += 376
                                Single_blood_proportion += 185
                                Risk_scores_proportion += 44

                            else:
                                predicted = evaluateBranch(
                                    modelsPickleList[15],
                                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment"],
                                    patient,
                                    modelsPickleList[17],
                                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk"],
                                    ind[10],
                                    ind[11]
                                )

                                if isinstance(predicted, float) and predicted < ind[10]:
                                    predicted_output[i] = predicted
                                    Questions_proportion += 703

                                elif isinstance(predicted, float) and predicted > ind[11]:
                                    predicted_output[i] = predicted
                                    Questions_proportion += 703
                                    Single_blood_proportion += 185
                                    Risk_scores_proportion += 44

                                #"Make it into else to make sure catch everything"
                                #if not isinstance(predicted_output[i], float) or predicted_output[i] == 0:
                                else:
                                    predicted = evaluateBranch(
                                        modelsPickleList[18],
                                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors"],
                                        patient,
                                        modelsPickleList[19],
                                        cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"],
                                        ind[12],
                                        ind[13]
                                    )

                                    if isinstance(predicted, float) and predicted < ind[12]:
                                        predicted_output[i] = predicted
                                        Questions_proportion += 756

                                    elif isinstance(predicted, float) and predicted > ind[13]:
                                        predicted_output[i] = predicted
                                        Questions_proportion += 756
                                        Single_blood_proportion += 185
                                        Risk_scores_proportion += 44

                                    else:
                                        predicted = evaluateBranch(
                                        modelsPickleList[19],
                                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors_oneDrop"],
                                        patient,
                                        modelsPickleList[20],
                                        cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"],
                                        ind[14],
                                        ind[15]
                                        )
                                        if isinstance(predicted, float) and predicted < ind[14]:
                                            predicted_output[i] = predicted
                                            Questions_proportion += 756
                                            Single_blood_proportion += 185
                                            
                                        else:
                                            predicted = predictIndividual(modelsPickleList[20], cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"], patient)[0]
                                            predicted_output[i] = predicted
                            
                                            Questions_proportion += 756
                                            Single_blood_proportion += 185
                                            Risk_scores_proportion += 44
            
            # Normalize proportions
        Questions_proportion /= (756 * len(dataset))
        Single_blood_proportion /= (185 * len(dataset))
        Risk_scores_proportion /= (44 * len(dataset))

        #Risk score as a median is the average of the lowest risk score (-3) and the highest (+9) so 3
        y_pred_binary = np.array(predicted_output) >= ind[15]
        y_true = np.array(actualValuesStatus)

        TP = np.sum((y_pred_binary == 1) & (y_true == 1))
        TN = np.sum((y_pred_binary == 0) & (y_true == 0))
        FP = np.sum((y_pred_binary == 1) & (y_true == 0))
        FN = np.sum((y_pred_binary == 0) & (y_true == 1))

        True_positive_rate = TP / (TP + FN) if (TP + FN) else 0
        True_negative_rate = TN / (TN + FP) if (TN + FP) else 0

        try:
            AUC = roc_auc_score(y_true, predicted_output)
        except ValueError:
            AUC = 0  

        try:
            from sksurv.metrics import concordance_index_censored
            y_surv = np.array([(bool(e), t) for e, t in zip(y_true, actualValuesFollowup)],
                              dtype=[('event', bool), ('time', float)])
            Cindex = concordance_index_censored(y_surv['event'], y_surv['time'], predicted_output)[0]
        except:
            Cindex = 0

        print(f"  1 - Questions proportion: {1 - Questions_proportion:.3f}")
        print(f"  1 - Single blood proportion: {1 - Single_blood_proportion:.3f}")
        print(f"  1 - Risk scores proportion: {1 - Risk_scores_proportion:.3f}")
        print(f"  True Positive Rate (TPR): {True_positive_rate:.3f}")
        print(f"  True Negative Rate (TNR): {True_negative_rate:.3f}")
        print(f"  AUC: {AUC:.3f}")
        print(f"  C-index: {Cindex:.3f}")


        goal_values = [0]*8
        goal_values[0] = 1 - Questions_proportion
        goal_values[1] = 1 - Single_blood_proportion
        goal_values[2] = 1 - Risk_scores_proportion
        goal_values[3] = True_positive_rate
        goal_values[4] = True_negative_rate
        goal_values[5] = AUC
        goal_values[6] = Cindex
        goal_values[7] = (
            self.goal_significance[0]*goal_values[0] +
            self.goal_significance[1]*goal_values[1] +
            self.goal_significance[2]*goal_values[2] +
            self.goal_significance[3]*goal_values[3] +
            self.goal_significance[4]*goal_values[4] +
            self.goal_significance[5]*goal_values[5] +
            self.goal_significance[6]*goal_values[6]
        ) / sum(self.goal_significance)

        return goal_values
        

    #This function is used by non_Dominated_Sol() to compare 
    # if a sol i.e. goal_value dominates another sol i.e. goal_value
    def dominate(self, solution1, solution2):
        check=2 # initialise solution comparison check to 2 i.e. equal
        ffs=len(solution1)
        dominate1=1
        equal1=1
        f=0
        while f<ffs and dominate1==1:
            if solution1[f]>solution2[f]:
                equal1=0
            elif solution1[f]==solution2[f]:
                do_nothing=1
            else:
                dominate1=0
            f=f+1
        
        if dominate1==1 and equal1==0:
            check=1
        elif dominate1==1 and equal1==1:
            check=2
        else:
            dominate2=1
            equal2=1
            f=0
            while f<ffs and dominate2==1:
                if solution2[f]>solution1[f]:
                    equal2=0
                elif solution2[f]==solution1[f]:
                    do_nothing=1
                else:
                    dominate2=0
                f=f+1
            
            if dominate2==1 and equal2==0:
                check=3
        
        return check

    # function to bucket solutions of a given generation into diff. pareto fronts  
    # This function is called by optimization()
    # adapted from script written for Missense SNPs ensemble_multiclassifier.py by Konstantinos Theofilatos 
    # A dominates B iff it is better than or equal to B on all dimensions and strictly better on at least one dimension
    #def non_Dominated_Sol(self,population,goal_values,individuals):
    # functioon inherited from ensemble_multiclassifier.py for Missense
    def getParetoFront(self,population,goal_values,stopIndx):
        assigned=0
        fronts=[0]*population # initialise pareto front for population size
        front=1
        #  create copy of goal_values so that original goal are not changed. Exclude weighted average
        eval_temp=copy.deepcopy(np.array(goal_values)) # change asarray to array to create copy
        eval_temp = eval_temp.T # take transpose so that goals are in rows and observations in columns
        
        #  reduce the metrics to PPV, sensitivity and ROC for Pareto Front. Align with choice of goal_significance weightage
        pareto_Index=[0,1,2,3,4,5,6] # no feature restriction, PPV, ROC, sensitivity
        
        eval_temp = [eval_temp[i] for i in pareto_Index]
        print ("Parento front computation")
        #print (eval_temp)
        
        number_of_solutions = copy.deepcopy(population) # deep copy of length of population
        
        #ref: Mishra, K. K., & Harit, S. (2010). A fast algorithm for finding the non-dominated set in multi objective optimization. International Journal of Computer Applications, 1(25), 35-39. 
        while assigned < population: # iterate untill non-dominated solutions assigned is less than population size
            non_dominated_solutions = [0]*number_of_solutions
            index=[i for i in range(len(eval_temp[0]))] # generate list of index based on count of goals
            self.Pareto_Wheel_Output += "Index for Ordered List %s: "%(index)  + "\n"
            
            eval_temp_index=list(zip(eval_temp[0],index)) # create list of tuples with goal val of sol. & index 
            # sort the goal values in decrease order i.e. highest comes 1st
            self.Pareto_Wheel_Output += "Tuple of goal and list index %s: "%(eval_temp_index)  + "\n"
            ordered_list= sorted(range(len(eval_temp_index)), key=lambda k: eval_temp_index[k],reverse=True) 
            
            non_dominated_solutions[0]=ordered_list[0]	# assign the 1st goal values as 1st non-dominanted sol.		
            number_of_non_dominated_solutions=1		# initiatlise the non-dominated sol count
            
            for i in range(1,number_of_solutions): # iterate from 1 to pop size as 1st i.e. 0 index is already selected
                n=0
                condition=0
                condition2=1
                while n<number_of_non_dominated_solutions and condition==0: # compare ordered list of solutions pairs 
                    solution1=[0]*(len(eval_temp)) # excludes weighted average goal & clasification model
                    solution2=[0]*(len(eval_temp)) # excludes weighted average goal & classification model
                    
                    for j in range(len(eval_temp)): # iterate on no of goals; compare solutions by goals
                        solution1[j]=eval_temp[j][ordered_list[i]]
                        solution2[j]=eval_temp[j][ordered_list[n]]				
                    
                    check=self.dominate(solution1,solution2) # compare the goal values in descending order
                    if check==3:
                        condition=1
                        condition2=0
                    elif check==1:
                        if number_of_non_dominated_solutions==1:
                            condition=1
                            non_dominated_solutions[0]=ordered_list[i]
                        else:
                            number_of_non_dominated_solutions=number_of_non_dominated_solutions-1	
                            del non_dominated_solutions[n]
                            
                    n=n+1
                    
                if condition2==1:
                    non_dominated_solutions[number_of_non_dominated_solutions]=ordered_list[i]
                    number_of_non_dominated_solutions=number_of_non_dominated_solutions+1
            
            sorted_non_dominated_solutions=sorted(non_dominated_solutions, reverse=True) # index sorting of non-dominated sol
            
            for i in range(number_of_non_dominated_solutions):
                assigned=assigned+1
                fronts[sorted_non_dominated_solutions[i]]=front
                for j in range(len(eval_temp)): # sets the chosen goals to zero
                    eval_temp[j][sorted_non_dominated_solutions[i]]=0		
            front=front+1					
        
        self.Pareto_Wheel_Output += "Calculated Pareto Frontiers %s "%(fronts)  + "\n\n"
        print("Pareto Front Calculation Completed")		
        
        ### check if front = 0 for error capture
        if min(fronts) == 0:
            f = open('FrontERROR.txt','w') # redirect all print statements to file output
            print ('Pareto front zero detected',file=f)
            # assign front 0 to be max(front)+1 so that it is worst sol
            newFront = max(fronts)+1
            for i,f in enumerate(fronts):
                if f==0: 
                    fronts[i]=newFront
            
        return fronts # returns list of pareto fronts for the observations i.e. population
	 
    #Tune fitness values by locating and using solution niches
    # adapted from script written for Missense SNPs ensemble_multiclassifier.py by Konstantinos Theofilatos 
    # uses distance for solutions within a given pareto front to see if they are similar or diff
    # and assigns a new fitness score
    def tuneFitnessValue(self,fronts,sigma_share,evaluation_values,rep,population,individuals):
        ####### evaluation values has last values as weighted average
        min_values = self.min_values
        max_values = self.max_values
        
        print("Starting Fitness Tuning As Per Pareto Frontiers")
        
        for i in range(1,max(fronts)+1): # start from 1st pareto front; index of fronts is starting with 1, increment by 1
            self.Pareto_Wheel_Output += "Fitness Tuning As Per Pareto Front: %s "%(i)  + "\n"
            #CHANGE 2
            ind = [y for y, x in enumerate(fronts) if x == i]


            self.Pareto_Wheel_Output += "Individuals in pareto front %s "%(ind)  + "\n"
            
            #Calculate max values per goal per pareto frontier			
            max_significances=[0]*(len(evaluation_values)-1) # exclude weighted average
            for j in range(len(ind)):
                for goal in range(len(evaluation_values)-1):
                    if evaluation_values[goal][ind[j]]>=max_significances[goal]:
                        max_significances[goal]=evaluation_values[goal][ind[j]]			
            
            self.Pareto_Wheel_Output += "Max. Significance %s "%(max_significances)  + "\n"
            
            # compute distance based similarity for the solutions within a given front
            for j in range(len(ind)): # iterate for indivdiuals within a given front
                m=0
                for k in range(len(ind)):
                    d=0
                    for gene in range(len(individuals[0])): 
                        d=d+((individuals[ind[j]][gene]-individuals[ind[k]][gene])/float(max_values[gene]-min_values[gene]))**2
                        
                    d=math.sqrt(d/(len(individuals[0])))
                    
                    if d<=sigma_share:
                        m=m+(1-((d/float(sigma_share))**2))
                    if m==0:
                        m=1
                self.Pareto_Wheel_Output += "Value of m  %s "%str(m)  + "\n"
                for goal in range(len(evaluation_values)-1):
                    evaluation_values[goal][ind[j]]=float(max_significances[goal])/m
        
        ## recalculate weighted sum of goals using goal_significance
        # goals are in rows and samples in columns
        for i in range(len(evaluation_values[0])): # compute weighted sum for each sample
            evaluation_values[-1][i]=0 # reset weighted average i.e. last row to zero
            for j in range(len(evaluation_values)-1): # exclude last row i.e. weighted sum in iteration
                evaluation_values[-1][i]=evaluation_values[-1][i]+evaluation_values[j][i]*self.goal_significance[j]
            evaluation_values[-1][i]=evaluation_values[-1][i]/float(len(evaluation_values)-1)
        
        ## Update Sum of weighted avg goal and avg of weighhted avg goal for the generation
        # to be used for visualisation
        sum_ranked=0
        for i in range(population):
            sum_ranked=sum_ranked+evaluation_values[-1][i]
        
        self.sum_ranked_eval_per_generation[rep]=sum_ranked
        self.average_ranked_eval_per_generation[rep]=sum_ranked/population
        
        print("Fitness Tuning As Per Pareto Frontiers Completed")
        
        return evaluation_values
    
    #roulett wheel based selection of ranked indivdiuals
    # adapted from script written for Missense SNPs ensemble_multiclassifier.py by Konstantinos Theofilatos 
    # evaluation values are of shape goals X samples
    def roulett_Wheel(self, evaluation_values_tuned,population,individuals,rep,bestSolIndex):
        
        min_values = self.min_values
        selected_individuals=[[0 for x in range(len(min_values))] for x in range(population)]
        indv_wheel = copy.deepcopy(individuals) # inclulded this to avoid any link between individuals and selected indv.
        # calculate the cumulative proportions i.e. selection probability based on weighted avg goal
        sum_prop=[0]*(population+1)
        for i in range(1, population+1): 
            sum_prop[i]=sum_prop[i-1]+evaluation_values_tuned[-1][i-1]/float(self.sum_ranked_eval_per_generation[rep])
        
        self.Pareto_Wheel_Output += "Cummuative proportions based on weighted avg post tuning of Fitness Value:  %s "%(sum_prop)  + "\n"
        
        for i in range(1, population): # get proportions from index 1 thru pop size as 0 is for best ind.
            random_number=random.uniform(0,1) # generate probability randomly
            for j in range(0,population): # select those indv. for which random no is within cummulative prop 
                if random_number>=sum_prop[j] and random_number<sum_prop[j+1]:
                    selected_individuals[i]=copy.deepcopy(indv_wheel[j]) # assign via deepcopy and not original
        
        ## assign best sol based on max of weighted sum of goals to index 0
        selected_individuals[0]= copy.deepcopy(indv_wheel[bestSolIndex]) # assign via deepcopy to avoid index 0 editing
        
        print("Roulette Wheel Selection Completed")
        
        return selected_individuals

    # function to apply two-point cross over
    # adapted from script written for Missense SNPs ensemble_multiclassifier.py by Konstantinos Theofilatos 
    def GACross_Over(self,population,cross_indv):
        

        
        temp_individuals=[[0 for x in range(len(self.min_values))] for x in range(population)] 
        
        cross_indv_copy = copy.deepcopy(cross_indv) # take deep copy to avoid index 0 editing
        features = copy.deepcopy(len(cross_indv_copy[0])) # take deep copy to avoid index 0 editing
        
        best_sol_before = sum(cross_indv[0]) # index 0 holds best sol
        
        for i in range(1,population-1,2): # 1st is best sol so preseve it
            random_number=random.uniform(0,1) # generate prob randomly 
            if random_number < self.twoCross_prob: 
                cross_point1=0
                cross_point2=0
                while cross_point1==cross_point2:
                    cross_point1 = math.ceil((features-4)*random.uniform(0,1)) # minus 4 to leave last positions
                    if cross_point1 < math.floor((2*features-1)/3): 
                        width = math.ceil(random.uniform(0,1)*(math.floor(features-1)/3 -2))
                        cross_point2 = cross_point1 + width
                    else:
                        width=math.ceil(random.uniform(0,1))*(math.floor(features/3 -1)-2-(cross_point1-math.floor(2*features/3)))
                        cross_point2=cross_point1+width
                if cross_point1>cross_point2:
                    temp_cross_point=cross_point1
                    cross_point1=cross_point2
                    cross_point2=temp_cross_point
                width=int(width)
                cross_point1=int(cross_point1)
                cross_point2=int(cross_point2)
                for j in range(cross_point1,cross_point2+1):
                    temp_individuals[i][j]=copy.deepcopy(cross_indv_copy[i+1][j])
                for j in range(cross_point1,cross_point2+1):
                    cross_indv_copy[i+1][j]=copy.deepcopy(cross_indv_copy[i][j])
                for j in range(cross_point1,cross_point2+1):
                    cross_indv_copy[i][j]=copy.deepcopy(temp_individuals[i][j])
            
            elif random_number>= self.twoCross_prob and random_number < (self.twoCross_prob + self.arithCross_prob): 
                # arithmetic cross over
                alpha=random.uniform(0,1)			
                for j in range(0,features): 
                    temp_individuals[i][j]=copy.deepcopy(alpha*cross_indv_copy[i][j]+(1-alpha)*cross_indv_copy[i+1][j])
                    temp_individuals[i+1][j]=copy.deepcopy((1-alpha)*cross_indv_copy[i][j]+(alpha)*cross_indv_copy[i+1][j])
                for k in range(0,features):				
                    cross_indv_copy[i][k]=copy.deepcopy(temp_individuals[i][k])
                    cross_indv_copy[i+1][k]=copy.deepcopy(temp_individuals[i+1][k])		
        
        best_sol_after = sum(cross_indv_copy[0]) # index 0 holds best sol; take after sum from copy version 
        
        ### check if best solution is preserved
        if (best_sol_before - best_sol_after) != 0.0:
            fcross = open('CrossOverERROR.txt','w') # redirect all print statements to file output
            self.output_message += "Cross over of best sol detected " + "\n"
            print ('Cross over for best solution detected. Sol sum delta %s'%(best_sol_before - best_sol_after),file=fcross)
        
        print("Crossover completed")
        return copy.deepcopy(cross_indv_copy)
    
    # function to apply adaptive mutation; Use deep copy for assignment to avoid index 0 editing
    # adapted from script written for Missense SNPs ensemble_multiclassifier.py by Konstantinos Theofilatos 
    # script edited for adaptive mutation rate
    def adaptiveMutation(self,population,mutate_indv, rep):
        print("Doing Adaptive Mutation")
        min_values = self.min_values
        max_values = self.max_values
        
        best_sol_before = sum(mutate_indv[0]) # index 0 holds best sol
        mutate_indv_copy = copy.deepcopy(mutate_indv) # take deep copy so that index 0 is not chnaged
        
        # get the mutation probability and gaussian variance proportion based on average similarity 
        # ref: Rapakoulia, T., Theofilatos, K., Kleftogiannis, D., Likothanasis, S., Tsakalidis, A., & Mavroudi, S. (2014). 
        # EnsembleGASVR: a novel ensemble method for classifying missense single nucleotide polymorphisms. Bioinformatics, 30(16), 2324-2333. 
        
        # compute average similarity between the members of the population and its best member i.e. at index 0
        a = np.array(mutate_indv_copy[:0:-1]) # take all elements from bottom to 1, exclude 0
        b = np.array([mutate_indv_copy[0]]) # best solution
        CDist= distance.cdist(a, b, 'chebyshev')
        AvgSimilarity = np.mean(CDist)/ (np.max(CDist) - np.min(CDist))
        self.Pareto_Wheel_Output += "Mutation Cal. Avg. Similarity %s "%(AvgSimilarity)  + "\n"
        
        numerator = float(self.mutation_prob[1]) - (1/self.population) # 0.2-(1/Pop size)
        delta = float(rep)*numerator/float(self.generations)
        if round(AvgSimilarity,2) >= round(self.avgSimThr,2): # high similaity, so increase the diversity with higher mutation
            mutationRate = float(self.mutation_prob[1]) + delta # 0.2+ delta
            gaussVarProp = float(self.gaussMut_varPro[1]) + delta # 0.5+ delta
        else:
            mutationRate = float(self.mutation_prob[1]) - delta # 0.2 - delta
            gaussVarProp = float(self.gaussMut_varPro[1]) - delta # 0.5 - delta
            if mutationRate < float(self.mutation_prob[0]): # reset if lesser than lower bound
                mutationRate = float(self.mutation_prob[0])
            if gaussVarProp < float(self.gaussMut_varPro[0]): # reset if lesser than lower bound
                gaussVarProp = float(self.gaussMut_varPro[0])
        
        self.Pareto_Wheel_Output += "Mutation Rate chosen %s "%(mutationRate)  + "\n"
        self.Pareto_Wheel_Output += "Gaussian Variance Proportion chosen %s "%(gaussVarProp)  + "\n"
        
        feature = copy.deepcopy(len(mutate_indv_copy[0]))
        #apply mutation operator
        for i in range(1,population): # preserve the 1st sol as its best
            for j in range(0,feature):
                random_number=random.uniform(0,1)
                if random_number<mutationRate:
                    # Gaussian distribution. mean zero and standard deviation = (max-min)*variance proportion
                    mutate_indv_copy[i][j]=copy.deepcopy(mutate_indv_copy[i][j]+random.gauss(0,gaussVarProp*(max_values[j]-min_values[j])))
            
            #Correct values out of boundaries
            for j in range(0, len(min_values)):
                if mutate_indv_copy[i][j] < min_values[j]:
                    mutate_indv_copy[i][j] = copy.deepcopy(min_values[j])
                if mutate_indv_copy[i][j] > max_values[j]:
                    mutate_indv_copy[i][j] = copy.deepcopy(max_values[j])
        
        best_sol_after = sum(mutate_indv_copy[0]) # index 0 holds best sol; take sum of index 0 from copy to check if any change
        
        ### check if best solution is preserved
        if (best_sol_before - best_sol_after) != 0.0:
            fmutate = open('MutationERROR.txt','w') # redirect all print statements to file output
            self.output_message += "Mutation of best sol in Gen %s"%rep + "\n"
            print ('Mutation of best solution detected. Sol sum delta %s'%(best_sol_before - best_sol_after),file=fmutate)
        
        return copy.deepcopy(mutate_indv_copy)
    
    ########################################################
    def labelPowerset (self):
        # take transpose of list of list i.e. the class labels outcome Y of shape [n_labels, n_sample]
        yNew = list(map(list, zip(*self.class_Labels))) # yNew shape [n_samples, n_labels]
        # convert to list of strings
        yNew = [[str(int(j)) for j in i] for i in yNew] # change int to str
        self.powerSetClass = list(map(''.join, yNew)) # list of strings with powerset of class labels given the dataset
    
    

    
    '''#This function pre-computes the wilconxon ranksum, JMIM and MRMR for given set of k values.
    # it then passes diff. populations of teh generation for evaluation to evaluate_individuals()
    # Pareto optimal solution search and cross-over + mutation for GA is also done here'''
    def optimization(self):
        
        
        self.output_message += "Parameters selected Gen: %s , Pop: %s"%(self.generations, 
                                                                                       self.population) + "\n" # capture the parameters 

        individuals = self.initialize() #Initialize Population of Individual Solutions
        
        #individuals = self.initializeBestCSV() # initialise with front1 solutions from previous run
        self.output_message += "Initial population formulated. Sample indivdiuals length : %s"%len(individuals[0]) + ", %s"%len(individuals[4]) + "\n"
        self.output_message += "Sample individual : %s"%(individuals[4]) + "\n"
        
        print("Individuals Initiatised")
        
        # variables for boundary condition / hold output of each generation
        generations = self.getGenerations()
                
        # make custom score f2 for cross_validate
        
        sigma_share=0.5/(float(len(individuals[0]))**(0.1)) # sigma for distance threshold
        
        #Apply Evolutionary Process to Optimize Individual Solutions
        print("Starting GA based optimisation")	
        premature_termination=0 
        front1_feat_perGen = [] # list of lists to hold front1 features per  generation
        front1_sol_perGen = [] # list of lists to hold front1 sol per  generation
        front1_index = 0 # index to write front1 sol per generartion
        final_selected_best = [] # list of lists to hold best sol features per generation
        best_index = 0 # index to write best sol per generartion
        average_per_generation = []
        for rep in range(generations): # iterate on generations
            print("\nGeneration:"+str(rep+1))
            # initialise goal values to zero, list of list with length = population size; calcualted by evaluate_individuals()
            # position 0 - Questions_proportion
            # position 1 - Single_blood_proportion
            # position 2 - Risk_scores_proportion
            # position 3 - True_positive_rate = Tp/(Tp+Fn)
            # position 4 - False_negative_rate = Tn /(Tn+Fp)
            # position 5 - AUC
            # position 6 - Cindex
            # position 7 - Weighted_Sum

                
        
            
            ############ Evaluate population of solutions. 
            self.output_message += "\n"
            self.output_message += "Generation : %s"%str(rep) + "\n"
            post_evaluate = False
            # evaluate individuals of a given population in paralle
            results = Parallel(n_jobs=7, backend="multiprocessing")(
            delayed(self.evaluate_individuals)(ind, dfTrain) for ind in individuals)

            self.goal_values = np.asarray(results)          # shape: (population, 8)
            goal_values = self.goal_values
            average_performance = 0 # average performance for convergence. initialise to zero
            convergence = 0.0
            avgAll = goal_values.mean(axis=0)               # average of each goal
            best_performance = goal_values.max(axis=0)      # max of each goal
            bestSolIndex = goal_values[:, 7].argmax()
            best_avg_perf = goal_values[bestSolIndex][7]
            average_performance=avgAll[7]
            self.output_message += "best sol index %s"%bestSolIndex + "\n"
            average_per_generation.append(average_performance)
            
            
            convergence = math.fabs(best_avg_perf - average_performance)
            print ("convergence value %s"%convergence)
            #Convergence criterion is checked in order to stop the evolution if the population is deemd us converged
            if convergence < self.convergeThreshold * average_performance:
                premature_termination=1			
            
            #Estimate pareto fronts for the solutions of this generation based on all goals except weighted average
            stopIndx = (len(goal_values[0])-1) # goals to slice for pareto front computation, excludes weighted average
            print ("Pareto stopIndx %s"%stopIndx)
            self.Pareto_Wheel_Output += "Generation : %s"%str(rep+1) + "\n"
            fronts = self.getParetoFront(len(individuals),goal_values,stopIndx)
            
            ##### save front1 solutions, features are intersect of GA individuals and filter method
            post_evaluate = True # when true this flag avoid classifier fitting within evaluate
            for_front1 = copy.deepcopy(individuals)
            for i,ind in enumerate(for_front1[:]): # iterate on solutions to pic sol with front = 1
                if fronts[i] == 1:
                    front1_sol_perGen.append([])
                    front1_sol_perGen[front1_index].append(rep) # add generation number
                    front1_sol_perGen[front1_index].extend(ind)
                    front1_feat_perGen.append([])
                    front1_feat_perGen[front1_index].append(rep) # add generation number
                    front1_feat_perGen[front1_index].extend(goal_values)
                    front1_index = front1_index + 1
            
            # get max per goal; and best sol
            self.max_eval_per_generation[rep] = list(map(max, zip(*[i for i in goal_values if i is not None])))
            self.output_message += "best performance goals :" + "\n"
            self.output_message += str(self.max_eval_per_generation[rep])
            self.output_message += "\n"
            
            self.max_sol_per_generation[rep] = copy.deepcopy(individuals[bestSolIndex]) # use index of bestSolIndex,there can be more than one index with max weighted average
            self.output_message += "best solution :" + "\n"
            self.output_message += str(self.max_sol_per_generation[rep])
            self.output_message += "\n"
            self.output_message += str(individuals[bestSolIndex])
            self.output_message += "\n"
                        
            #Check if its last generation, do not apply pareto, roulette wheel, cross over and mutation
            if rep == (generations-1) or premature_termination==1:
                if premature_termination ==0:
                    print ("This was last generation without convergence")
                    self.output_message += "No Convergence" + "\n"
                else:
                    print ("Solution converged")
                    self.output_message += "Convergence at Generation : %s"%str(rep+1) + "\n"
                
                #np.savetxt(pathFeature + os.sep +"goals_Final.csv", goal_values, delimiter=",", header = self.goal_header)
                
            
            # compute fitness value guided by grouping of sol in each pareto fronts and recompute weighthed average
            # so retain weighted average goal
            stopIndxAvg = (len(goal_values[0])) # last column is classification model so exclude that
            
            evaluation_values = copy.deepcopy(np.asarray(goal_values)) 
            evaluation_values = evaluation_values.T
            evaluation_values_tuned = self.tuneFitnessValue(fronts,sigma_share,evaluation_values,rep,len(individuals),individuals)
            
            # routleet wheel selection on tuned evaluation values by patreto front
            wheel_indv = copy.deepcopy(individuals) #take a deep copy so that best sol is not altered
            selected_individuals = self.roulett_Wheel(evaluation_values_tuned,len(individuals),wheel_indv,rep,bestSolIndex)
            
            # do cross-over 
            cross_indv = copy.deepcopy(selected_individuals) # take deep copy so that best ind is not altered; index 0 solution
            cross_individuals = self.GACross_Over(len(individuals),cross_indv)
            # do muttaion
            mutate_indv = copy.deepcopy(cross_individuals) # take deep copy so that best ind is not altered; index 0 solution
            mutated_individuals = self.adaptiveMutation(len(individuals),mutate_indv,rep)
            
            
            individuals=copy.deepcopy(mutated_individuals)
        
        # last solution is the final solution 
        try:
            dfFinalSol = pd.DataFrame(individuals)
            #SADIQ Got rid of the header
            dfFinalSol.to_csv(path_or_buf = pathFeature + os.sep+"FinalSolutionsAll.csv", index=False, header=False)
        except ValueError as e:
            dfFinalSol = pd.DataFrame(individuals)
            dfFinalSol.to_csv(path_or_buf = pathFeature + os.sep+"FinalSolutionsAll.csv", index=False, header=False)
        
        front1FinalIndv = []
        front1FinalGoal = []
        print("goal values")
        print(goal_values)
        index = 0
        for_final_front1 = copy.deepcopy(individuals)
        for i,ind in enumerate(for_final_front1[:]):
                if fronts[i] == 1:
                    front1FinalIndv.append([])
                    front1FinalIndv[index].extend(ind)
                    
                    front1FinalGoal.append([])
                    front1FinalGoal[index].extend(goal_values[i])

                    index = index + 1
        
        solHeader = ['Min_Threshold1','Max_Threshold1','Min_Threshold2','Max_Threshold2','Min_Threshold3','Max_Threshold3','Min_Threshold4','Max_Threshold4','Min_Threshold5','Max_Threshold5','Min_Threshold6','Max_Threshold6','Min_Threshold7','Max_Threshold7','Min_Threshold8','Classification_Threshold']
        try:
            df = pd.DataFrame(front1FinalIndv)
            df.to_csv(path_or_buf = pathFeature + os.sep+"FinalSolFront1.csv", index=False, header=solHeader)
        except ValueError as e:
            df = pd.DataFrame(front1FinalIndv)
            df.to_csv(path_or_buf = pathFeature + os.sep+"FinalSolFront1.csv", index=False, header=False)
        ##### ValueError: Writing 0 cols but got 428 aliases
        np.savetxt(pathFeature + os.sep +"goals_FinalFront1.csv", front1FinalGoal, delimiter=",", header = self.goal_header)
        df=pd.DataFrame(self.max_eval_per_generation)
        df.to_csv(path_or_buf = pathFeature + os.sep+ 'Max Solution Performance Per Generation.csv', index=False, header=False)
        df=pd.DataFrame(average_per_generation)
        df.to_csv(path_or_buf = pathFeature + os.sep+ 'Average Performance Per Generation.csv' , index=False, header=False)
        return dfTrain, front1FinalIndv
        
    ##################################################################################
    # function to finalise and save FINAL Model for prediction 
    def FinaliseAndSaveModel(self,dataTran, front1FinalIndv,front1FinalFeature):
        print ("Fitting and saving final model(s) for prediction")
        ###### Finalizing the model : train the model on the entire training dataset available
        finalFlag = True # set the flag for final solution True so that CV is not done when training the classifiers
        skip_count = 0 # check how many models with 0 features skipped
        chain_dictionary = [] # initialise list for writing final front1 classifier chain. List allows same chain to be associated with more than one model
        # start the loop post optimisation length of optimisation parameters
        for ind in range(len(front1FinalIndv)): # iterate on all sol selected
            # train the classifier ; front1FinalIndv[ind][0] = classifier_type; front1FinalIndv[ind][1] = C_val; 
            # front1FinalIndv[ind][2] = Gamma_val; front1FinalIndv[ind][7] = Treelevel; front1FinalIndv[ind][0] = TreeCount,ftwo_scorer = 0
            if len(front1FinalFeature[ind])>0: # check for zero features
            
                if len(MULTI_LABELS) == 1: # for Single label
                    try:
                        clf,X, Y = self.train_Classifier(dataTran,front1FinalFeature[ind],front1FinalIndv[ind][0],front1FinalIndv[ind][1],
                                             front1FinalIndv[ind][2], front1FinalIndv[ind][7], front1FinalIndv[ind][8],0,finalFlag,copy.deepcopy(MULTI_LABELS))
                        clf.fit(X,Y)
                        dump(clf, pathModel + os.sep + str(ind)+'finalSingleModel.pkl')
                        
                    except ValueError as e: # ValueError: Found array with 0 feature(s) while a minimum of 1 is required.
                        skip_count = skip_count + 1
                        continue
                
                else: # train classifier chain for multi label
                    try:
                        models_list, inputs, outputs  = self.train_Classifier_MultiLabel(dataTran,front1FinalFeature[ind],front1FinalIndv[ind][0],front1FinalIndv[ind][1],
                                             front1FinalIndv[ind][2], front1FinalIndv[ind][7], front1FinalIndv[ind][8],0,finalFlag,copy.deepcopy(MULTI_LABELS))
                        dump(models_list, pathModel + os.sep + str(ind)+'finalChainOfModels.pkl')
                        
                    except ValueError as e: # ValueError: Found array with 0 feature(s) while a minimum of 1 is required.
                        skip_count = skip_count + 1
                        continue
                    
                    # writing the classifier chain as dictionary to be used by prediction module
                    chain_dictionary.append([])
                    chain_dictionary[ind].extend(self.dict_chain[int(front1FinalIndv[ind][7])])
        
        if len(MULTI_LABELS) == 1:
            chain_dictionary.append(MULTI_LABELS)
        
        df = pd.DataFrame(chain_dictionary)
        df.to_csv(path_or_buf = pathModel + os.sep+"classifierChain.csv",index=False,header=False)
        
        print ("Final model(s) saving completed and %s with 0 features skipped"%skip_count)

def readArguments(argv):
    # Get parameters based on command line input 
    parser = argparse.ArgumentParser()
    #parser.add_argument('file', metavar='DATA_FILENAME', help='txt file name with input data') # default type is str
    parser.add_argument('-pp','--population', nargs='?', type = int, default=300, help='Population size, default is 300')
    parser.add_argument('-g','--generations', nargs='?', type = int, default=100, help='Count of max generations, default is 100')
    parser.add_argument('-f','--folds', nargs='?', type = int, default=5, help='Count of folds for cross validation, default is 5')
    
    args = parser.parse_args() # get the mandatory and default arguments
    
    return args

################### main called when ensemble_cvd.py is run
def mainFunction(varDict):
    parser = argparse.ArgumentParser()
    
    #min_values=[0.01,0.0001,0.01,0.01,0.00005,1.0,2.0, 0.0,10.0]
    #max_values=[2.9,10.0,1.0,3.9,0.05,20.0,10.0, 1.0,300.0]

    parser.add_argument('-pp','--population', nargs='?', type = int, default=300, help='Population size, default is 300')
    parser.add_argument('-g','--generations', nargs='?', type = int, default=100, help='Count of max generations, default is 100')
    #parser.add_argument('-f','--folds', nargs='?', type = int, default=5, help='Count of folds for cross validation, default is 5')
   
    categories=["Early life factors_Early life factors","Family history_Family history","Health and medical history_Breathing","Health and medical history_Cancer screening","Health and medical history_Chest pain","Health and medical history_Claudication and peripheral artery disease","Health and medical history_Eyesight","Health and medical history_General health","Health and medical history_Hearing","Health and medical history_Medical conditions","Health and medical history_Medication","Health and medical history_Mouth","Health and medical history_Operations","Health and medical history_Pain","Lifestyle and environment_Alcohol","Lifestyle and environment_Diet","Lifestyle and environment_Electronic device use","Lifestyle and environment_Physical activity","Lifestyle and environment_Sexual factors","Lifestyle and environment_Sleep","Lifestyle and environment_Smoking","Lifestyle and environment_Sun exposure","Psychosocial factors_Mental health","Psychosocial factors_Socal support","Sex-specific factors","Sociodemographics_Early life factors","Sociodemographics_Education","Sociodemographics_Employment","Sociodemographics_Ethnicity","Sociodemographics_Household","Sociodemographics_Other sociodemographic factors"]
    
   
    ########################## perform adaptive GA based feature selection
    #size of chromosome 8X2
    evaluate_values = 16
        
    # pass dataset and dataTran to feature selection as ranksum uses dataformat of shape dataset while others use dataTran format
    optimizer = DiagnosticOptimization(dfTrain,dfTestFeaturesCols, dfTrain["CVD_followup"], dfTest, dfTest["CVD_followup"], evaluate_values,varDict['-pp'],varDict['-g'])
    trainingData,front1FinalIndv = optimizer.optimization() 
    dataset = dfTest
    goal_values = np.zeros([len(front1FinalIndv), 8])
    iter=0
    for ind in front1FinalIndv:
        Questions_proportion = 0
        Single_blood_proportion = 0 
        Risk_scores_proportion = 0 
        predicted_output = [0 for _ in range(len(dataset))]

        actualValuesStatus = []
        actualValuesFollowup = []

        for i in range(len(dfTest)):
            patient = dataset.iloc[[i]]
            actualValuesStatus.append(dataset['CVD_status'].iloc[i])
            actualValuesFollowup.append(dataset['CVD_followup'].iloc[i])

            predicted = evaluateBranch(
                modelsPickleList[0],
                cleaned_model_to_csvs["Sociodemographics"],
                patient,
                modelsPickleList[2],
                cleaned_model_to_csvs["Sociodemographics_oneDrop_qrisk"],
                ind[0],
                100
            )

            if isinstance(predicted, float) and predicted < ind[0]:
                predicted_output[i] = predicted
                Questions_proportion += 97


            elif isinstance(predicted, float) and predicted > ind[1]:
                predicted_output[i] = predicted
                Questions_proportion += 97
                Single_blood_proportion += 185
                Risk_scores_proportion += 44

            else:
                predicted = evaluateBranch(
                    modelsPickleList[3],
                    cleaned_model_to_csvs["Sociodemographics_Health and medical history"],
                    patient,
                    modelsPickleList[5],
                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_oneDrop_qrisk"],
                    ind[2],
                    ind[3]
                )

                if isinstance(predicted, float) and predicted < ind[2]:
                    predicted_output[i] = predicted
                    Questions_proportion += 242

                elif isinstance(predicted, float) and predicted > ind[3]:
                    predicted_output[i] = predicted
                    Questions_proportion += 242
                    Single_blood_proportion += 185
                    Risk_scores_proportion += 44

                else:
                    predicted = evaluateBranch(
                        modelsPickleList[6],
                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors"],
                        patient,
                        modelsPickleList[8],
                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_oneDrop_qrisk"],
                        ind[4],
                        ind[5]
                    )

                    if isinstance(predicted, float) and predicted < ind[4]:
                        predicted_output[i] = predicted
                        Questions_proportion += 312


                    elif isinstance(predicted, float) and predicted > ind[5]:
                        predicted_output[i] = predicted
                        Questions_proportion += 312
                        Single_blood_proportion += 185
                        Risk_scores_proportion += 44

                    else:
                        predicted = evaluateBranch(
                            modelsPickleList[9],
                            cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors"],
                            patient,
                            modelsPickleList[11],
                            cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_oneDrop_qrisk"],
                            ind[6],
                            ind[7]
                        )

                        if isinstance(predicted, float) and predicted < ind[6]:
                            predicted_output[i] = predicted
                            Questions_proportion += 319


                        elif isinstance(predicted, float) and predicted > ind[7]:
                            predicted_output[i] = predicted
                            Questions_proportion += 319
                            Single_blood_proportion += 185
                            Risk_scores_proportion += 44

                        else:
                            predicted = evaluateBranch(
                                modelsPickleList[12],
                                cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history"],
                                patient,
                                modelsPickleList[14],
                                cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_oneDrop_qrisk"],
                                ind[8],
                                ind[9]
                            )

                            if isinstance(predicted, float) and predicted < ind[8]:
                                predicted_output[i] = predicted
                                Questions_proportion += 376


                            elif isinstance(predicted, float) and predicted > ind[9]:
                                predicted_output[i] = predicted
                                Questions_proportion += 376
                                Single_blood_proportion += 185
                                Risk_scores_proportion += 44

                            else:
                                predicted = evaluateBranch(
                                    modelsPickleList[15],
                                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment"],
                                    patient,
                                    modelsPickleList[17],
                                    cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk"],
                                    ind[10],
                                    ind[11]
                                )

                                if isinstance(predicted, float) and predicted < ind[10]:
                                    predicted_output[i] = predicted
                                    Questions_proportion += 703

                                elif isinstance(predicted, float) and predicted > ind[11]:
                                    predicted_output[i] = predicted
                                    Questions_proportion += 703
                                    Single_blood_proportion += 185
                                    Risk_scores_proportion += 44

                                #"Make it into else to make sure catch everything"
                                #if not isinstance(predicted_output[i], float) or predicted_output[i] == 0:
                                else:
                                    predicted = evaluateBranch(
                                        modelsPickleList[18],
                                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors"],
                                        patient,
                                        modelsPickleList[19],
                                        cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"],
                                        ind[12],
                                        ind[13]
                                    )

                                    if isinstance(predicted, float) and predicted < ind[12]:
                                        predicted_output[i] = predicted
                                        Questions_proportion += 756

                                    elif isinstance(predicted, float) and predicted > ind[13]:
                                        predicted_output[i] = predicted
                                        Questions_proportion += 756
                                        Single_blood_proportion += 185
                                        Risk_scores_proportion += 44

                                    else:
                                        predicted = evaluateBranch(
                                        modelsPickleList[19],
                                        cleaned_model_to_csvs["Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_Psychosocial factors_oneDrop"],
                                        patient,
                                        modelsPickleList[20],
                                        cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"],
                                        ind[14],
                                        ind[15]
                                        )
                                        if isinstance(predicted, float) and predicted < ind[14]:
                                            predicted_output[i] = predicted
                                            Questions_proportion += 756
                                            Single_blood_proportion += 185
                                        else:
                                            predicted = predictIndividual(modelsPickleList[20], cleaned_model_to_csvs["Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk"], patient)[0]
                                            predicted_output[i] = predicted
                            
                                            Questions_proportion += 756
                                            Single_blood_proportion += 185
                                            Risk_scores_proportion += 44
        


        # Normalize proportions
        Questions_proportion /= (756 * len(dataset))
        Single_blood_proportion /= (185 * len(dataset))
        Risk_scores_proportion /= (44 * len(dataset))

        #Risk score as a median is the average of the lowest risk score (-3) and the highest (+9) so 3
        y_pred_binary = np.array(predicted_output) >= ind[15]
        y_true = np.array(actualValuesStatus)

        TP = np.sum((y_pred_binary == 1) & (y_true == 1))
        TN = np.sum((y_pred_binary == 0) & (y_true == 0))
        FP = np.sum((y_pred_binary == 1) & (y_true == 0))
        FN = np.sum((y_pred_binary == 0) & (y_true == 1))

        True_positive_rate = TP / (TP + FN) if (TP + FN) else 0
        True_negative_rate = TN / (TN + FP) if (TN + FP) else 0

        try:
            AUC = roc_auc_score(y_true, predicted_output)
        except ValueError:
            AUC = 0  

        try:
            from sksurv.metrics import concordance_index_censored
            y_surv = np.array([(bool(e), t) for e, t in zip(y_true, actualValuesFollowup)],
                              dtype=[('event', bool), ('time', float)])
            Cindex = concordance_index_censored(y_surv['event'], y_surv['time'], predicted_output)[0]
        except:
            Cindex = 0

        print(f"Individual {ind}:")
        print(f"  1 - Questions proportion: {1 - Questions_proportion:.3f}")
        print(f"  1 - Single blood proportion: {1 - Single_blood_proportion:.3f}")
        print(f"  1 - Risk scores proportion: {1 - Risk_scores_proportion:.3f}")
        print(f"  True Positive Rate (TPR): {True_positive_rate:.3f}")
        print(f"  True Negative Rate (TNR): {True_negative_rate:.3f}")
        print(f"  AUC: {AUC:.3f}")
        print(f"  C-index: {Cindex:.3f}")


        # Set goal values
        goal_values[iter][0] = 1 - Questions_proportion
        goal_values[iter][1] = 1 - Single_blood_proportion
        goal_values[iter][2] = 1 - Risk_scores_proportion
        goal_values[iter][3] = True_positive_rate
        goal_values[iter][4] = True_negative_rate
        goal_values[iter][5] = AUC
        goal_values[iter][6] = Cindex
        iter+=1
        df = pd.DataFrame(goal_values)
        df.to_csv(path_or_buf = pathFeature + "testSetGoalVals.csv", index=False, header=False) # pval data






In [ ]:
#G-100 pp 30
varDict={"-g": 100, "-pp":30}
mainFunction(varDict) # pass the dictionary of arguments to main

# run the code using command below
# python multi_objective_optimization.py -g 5 -p 10

Individuals Initiatised
Starting GA based optimisation

Generation:1
  1 - Questions proportion: 0.057
  1 - Single blood proportion: 0.105
  1 - Risk scores proportion: 0.106
  True Positive Rate (TPR): 0.717
  True Negative Rate (TNR): 0.703
  AUC: 0.777
  C-index: 0.771
  1 - Questions proportion: 0.021
  1 - Single blood proportion: 0.043
  1 - Risk scores proportion: 0.056
  True Positive Rate (TPR): 0.026
  True Negative Rate (TNR): 0.998
  AUC: 0.777
  C-index: 0.772
  1 - Questions proportion: 0.000
  1 - Single blood proportion: 0.000
  1 - Risk scores proportion: 0.000
  True Positive Rate (TPR): 0.010
  True Negative Rate (TNR): 1.000
  AUC: 0.778
  C-index: 0.773
  1 - Questions proportion: 0.042
  1 - Single blood proportion: 0.210
  1 - Risk scores proportion: 0.210
  True Positive Rate (TPR): 0.972
  True Negative Rate (TNR): 0.214
  AUC: 0.776
  C-index: 0.771
  1 - Questions proportion: 0.045
  1 - Single blood proportion: 0.060
  1 - Risk scores proportion: 0.060
  Tr

In [30]:
!ls

MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_Sex-specific_factors_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_Health_and_medical_history_oneDrop_qrisk.pkl
MRMR_COX_FULL_MODEL_+QRISKSociodemographics_oneDrop_qrisk.pkl
MRMR_COX_Sociodemographics.pkl
MRMR_COX_Sociodemographics_Health_and_medical_history.pkl
MRMR_COX_Sociode

In [31]:
!ls Output_FeatSelect

2025_06_02  2025_06_02testSetGoalVals.csv


In [32]:
!ls Output_FeatSelect/2025_06_02

'Average Performance Per Generation.csv'
 FinalSolFront1.csv
 FinalSolutionsAll.csv
'Max Solution Performance Per Generation.csv'
 goals_FinalFront1.csv


In [34]:
FinalSolFront1 = pd.read_csv('Output_FeatSelect/2025_06_02/FinalSolFront1.csv')
FinalSolFront1

,Min_Threshold1,Max_Threshold1,Min_Threshold2,Max_Threshold2,Min_Threshold3,Max_Threshold3,Min_Threshold4,Max_Threshold4,Min_Threshold5,Max_Threshold5,Min_Threshold6,Max_Threshold6,Min_Threshold7,Max_Threshold7,Min_Threshold8,Classification_Threshold
0,-2.484193,5.000000,-0.947774,5.000000,1.000000,6.276615,1.000000,5.620068,1.000000,8.681749,1.000000,8.748657,1.000000,5.000000,1.000000,1.792720
1,-1.563720,6.944407,-1.888616,8.209462,0.864605,5.762497,-0.211366,5.931304,-1.340304,5.000000,-0.209072,7.760002,-2.500000,5.000000,0.022641,3.074763
2,-2.500000,7.055593,-0.153862,6.418866,0.454577,5.000000,-1.251862,5.000000,0.040180,6.080559,-0.229976,7.746080,-1.302351,5.637896,0.485388,3.112052
3,-0.799507,7.957415,-2.500000,8.798075,-1.517770,6.145288,0.116261,8.241050,-1.815650,9.000000,-0.417373,5.157499,-0.517976,5.222527,1.000000,1.638921
4,-0.703108,8.402900,0.573783,5.063458,-0.895456,5.655919,1.000000,9.000000,-1.305070,7.508770,0.454413,7.108318,-2.500000,5.127444,-0.853306,2.547014
5,1.000000,6.624750,-0.988286,8.351893,1.000000,5.000000,0.012210,5.000000,1.000000,8.375118,-0.068469,9.000000,-1.533015,9.000000,1.000000,4.014643
6,-0.231252,5.000000,-2.500000,7.556734,-1.552548,8.048886,-1.412026,9.000000,-0.555866,8.464159,-1.252060,5.605640,-1.408749,8.784983,1.000000,0.230816
7,1.000000,5.000000,-0.947774,5.000000,0.388595,5.000000,0.568337,9.000000,1.000000,7.277051,-2.500000,7.457110,-2.173024,5.309422,-1.433755,0.000000
8,-2.484193,5.923599,-2.500000,9.000000,1.000000,5.000000,1.000000,5.620068,1.000000,8.681749,1.000000,8.748657,1.000000,5.000000,1.000000,0.000000
9,0.921640,8.612685,-1.406337,8.929677,1.000000,8.649072,0.731134,8.850317,-2.500000,5.000000,-1.154680,8.576633,0.885723,5.355557,1.000000,2.341229


In [35]:
FinalSolFront1 = pd.read_csv('Output_FeatSelect/2025_06_02testSetGoalVals.csv')
FinalSolFront1

,0.11486744730180898,0.5689328918023924,0.5769835101709806,0.6562675135438072,0.7235441627194205,0.7455677952387572,0.7413399295453313,0.0
0,0.055988,0.137655,0.167813,0.163647,0.972295,0.762418,0.757744,0.0
1,0.028122,0.127703,0.257181,0.152625,0.974849,0.762851,0.758164,0.0
2,0.010237,0.097773,0.422932,0.741640,0.653478,0.763039,0.758336,0.0
3,0.076793,0.321552,0.321552,0.345694,0.910690,0.757485,0.752787,0.0
4,0.086108,0.176624,0.424047,0.032692,0.998022,0.759412,0.754929,0.0
5,0.000918,0.010230,0.384174,0.970297,0.203511,0.763904,0.759185,0.0
6,0.079439,0.142453,0.142453,0.995890,0.028866,0.762253,0.757617,0.0
7,0.110988,0.568933,0.573765,0.989539,0.051844,0.745583,0.741368,0.0
8,0.066458,0.519844,0.535869,0.440501,0.869332,0.748079,0.743748,0.0
9,0.081224,0.190697,0.190697,0.987110,0.061679,0.761392,0.756813,0.0


In [36]:
FinalSolFront1 = pd.read_csv("Output_FeatSelect/2025_06_02/Max Solution Performance Per Generation.csv")
FinalSolFront1

,0.08703116284632584,0.49049809961992397,0.50250050010002,0.9919028340080972,0.9997780244173141,0.7780581180604547,0.7726068332403987,0.5548286453274796
0,0.088608,0.563113,0.568114,0.949393,0.999778,0.777972,0.772556,0.561279
1,0.072178,0.563513,0.572515,0.997976,0.999778,0.778359,0.772929,0.569618
2,0.081535,0.563513,0.572515,0.983806,0.999556,0.778731,0.773517,0.587727
3,0.099436,0.563113,0.563113,0.985830,0.999778,0.778215,0.772792,0.587727
4,0.081407,0.563113,0.563113,0.987854,0.999778,0.778042,0.772660,0.587727
...,...,...,...,...,...,...,...,...
94,0.127607,0.567313,0.575915,0.997976,0.999778,0.777981,0.772576,0.599030
95,0.127607,0.567313,0.575915,0.995951,0.999556,0.777156,0.771666,0.599030
96,0.127607,0.567313,0.575915,0.995951,0.999778,0.777458,0.772052,0.599030
97,0.127607,0.567313,0.576115,0.991903,0.999778,0.777946,0.772545,0.599030


In [37]:
FinalSolFront1 = pd.read_csv("Output_FeatSelect/2025_06_02/Average Performance Per Generation.csv")
FinalSolFront1

,0.4502567338852205
0,0.460779
1,0.457765
2,0.472083
3,0.467354
4,0.479878
...,...
94,0.521935
95,0.518185
96,0.505430
97,0.502827


In [38]:
import subprocess

upload_cmd = [
    "dx", "upload", "Output_FeatSelect",
    "--path", "SADIQ/genetic algorithm/solutions/",
    "--recursive"
]

result = subprocess.run(upload_cmd, capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)
print("Return Code:", result.returncode)


STDOUT:
 
STDERR:
 
Return Code: 0


In [39]:
import subprocess

upload_cmd = [
    "dx", "upload", "Output_PreProcess",
    "--path", "SADIQ/genetic algorithm/solutions/",
    "--recursive"
]

result = subprocess.run(upload_cmd, capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)
print("Return Code:", result.returncode)
#Output_Model
#Output_PreProcessls

STDOUT:
 
STDERR:
 
Return Code: 0


In [ ]:
print("Hello")

In [ ]:
!ls Output_Model

In [40]:
import subprocess

upload_cmd = [
    "dx", "upload", "Output_Model",
    "--path", "SADIQ/genetic algorithm/solutions/",
    "--recursive"
]

result = subprocess.run(upload_cmd, capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)
print("Return Code:", result.returncode)
#Output_Model
#Output_PreProcessls

STDOUT:
 
STDERR:
 
Return Code: 0
